In [1]:
import numpy as np

Initialize: b, N, the list of regimes of length m, the transitions probabilities


Create a function that takes the list of regimes of time N, and a list of li, the initial price So, the step volatility sigma  and finds out the prices at each step.

To do so we need:

    - The list of all probabilities corresponding to up, down, middle at each step, in consideration of all the regimes
    - compute the prices using the formula

In [13]:
b= 4
M = list(range(-b,b+1))
m = 4
R = range(1,m+1)
N= 100

Pa = np.random.rand(m,m)
print(Pa.sum(axis=1, keepdims=True))
Pa = Pa/Pa.sum(axis=1, keepdims=True)
Pa

[[1.54544311]
 [0.95945225]
 [2.02416643]]


array([[0.62120672, 0.27946447, 0.09932881],
       [0.02007414, 0.57036964, 0.40955622],
       [0.33285354, 0.17462109, 0.49252537]])

In [ ]:
sigma = np.array([0.9,0.5,0.7,0.2])
r = np.array([0.02,0.1,0.06,0.15])

: 

In [ ]:
def prices(regimes,L,moves,So,sigma_):


    return None

def nodes_probablities(regimes,moves):

    
    return None